In [ ]:
# Update the below line with the path to your SigKit repository location
%cd /workspaces/SigKit

%pip install -e .

In [ ]:
"""
Example of using the SigKit library to create a signal, load the pretrained model,
and run inference on the signal to classify its modulation type.

NOTE: Make sure to pull down lfs objects before running this example.
```bash
git lfs install
git lfs pull
```
"""
import numpy as np
import torch

from sigkit.core.base import Signal
from sigkit.modem.psk import PSK
from sigkit.models.Module import SigKitClassifier
from sigkit.transforms.utils import InferenceTransform
from sigkit.models.utils import PSK_CLASS_MAP

In [ ]:
"""Create a Modulated Signal from Bits"""

sample_rate = 1024 # Sample rate in Hz
symbol_rate = 32 # Symbol rate
carrier_frequency = 128 # Carrier frequency in Hz, 0 is baseband

# NOTE: All pytorch operations expect baseband, 0 is our default, we set a cf here
# for visualizations later.

modem = PSK(
    sample_rate=sample_rate,
    symbol_rate=symbol_rate,
    n_components=4, # Number of PSK components (e.g., 4 for QPSK)
    cf=carrier_frequency
)

num_symbols = (4096 // modem.sps) # Model expects 4096 samples
bits = np.random.randint(
    0, 2, size=num_symbols * modem.bits_per_symbol, dtype=np.uint8
)
signal: torch.Tensor = modem.modulate(bits).to_tensor().to("cpu")


In [ ]:
"""Load the Pretrained Model and Run Inference."""
model = SigKitClassifier.load_from_checkpoint("./data/checkpoints/best.ckpt", num_classes=6).to("cpu")
model.eval()

with torch.no_grad():
    signal = InferenceTransform(signal) # Apply necessary transform to normalize and reshape
    output = model(signal.unsqueeze(0))  # Add batch dimension
    predicted_class = torch.argmax(output, dim=1).item()

print(f"Predicted Modulation Class: {predicted_class}")
print(f"Class Name: {PSK_CLASS_MAP[predicted_class]}")